In [21]:
import warnings
from pathlib import Path
from loguru import logger
from datetime import datetime
from typing import Optional, Union
from itertools import chain

from astropy.io import fits
from astropy.io import votable
from astropy.io.votable import tree

import astropy.wcs as wcs
from astropy.coordinates import SkyCoord

import astropy.units as u
from astropy.nddata.utils import Cutout2D
from astropy.time import Time
from mocpy import MOC, STMOC

from vast_post_processing import crop

DATA_PATH = (Path(".") / "test-data" / "TILES").resolve()
IMAGE_PATH = (
    DATA_PATH
    / "STOKESI_IMAGES"
    / "epoch_38"
    / "image.i.VAST_0334-37.SB50801.cont.taylor.0.restored.conv.fits"
)
VOTABLE_PATH = (
    DATA_PATH
    / "STOKESI_SELAVY"
    / "epoch_38"
    / "selavy-image.i.VAST_0334-37.SB50801.cont.taylor.0.restored.conv.components.xml"
)

In [22]:
hdu = fits.open(IMAGE_PATH)[0]
field_centre = crop.get_field_centre(hdu.header)
cropped_hdu = crop.crop_hdu(hdu, field_centre)
vot = votable.parse(VOTABLE_PATH)
cropped_table = crop.crop_catalogue(vot, cropped_hdu)

2023-08-22 16:26:10.116 | DEBUG    | vast_post_processing.crop:get_field_centre:28 - Finding field centre
2023-08-22 16:26:10.125 | DEBUG    | vast_post_processing.crop:get_field_centre:35 - <SkyCoord (FK5: equinox=2000.0): (ra, dec) in deg
    (53.61702083, -37.63907778)>
2023-08-22 16:26:10.128 | DEBUG    | vast_post_processing.crop:crop_hdu:49 - Cropping HDU
2023-08-22 16:26:10.695 | DEBUG    | vast_post_processing.crop:crop_catalogue:71 - Cropping catalogue


In [23]:
moc = crop.wcs_to_moc(cropped_hdu)
stmoc = crop.moc_to_stmoc(moc, hdu)
print(stmoc)

2023-08-22 16:26:10.740 | DEBUG    | vast_post_processing.crop:wcs_to_moc:86 - Creating MOC


KeyError: "Keyword 'DATE-BEG' not found."

In [24]:
for key in hdu.header:
    print(key)

SIMPLE
BITPIX
NAXIS
NAXIS1
NAXIS2
EXTEND
BSCALE
BZERO
BUNIT
BTYPE
EQUINOX
RADESYS
LONPOLE
LATPOLE
PC1_1
PC2_1
PC3_1
PC4_1
PC1_2
PC2_2
PC3_2
PC4_2
PC1_3
PC2_3
PC3_3
PC4_3
PC1_4
PC2_4
PC3_4
PC4_4
CTYPE1
CRVAL1
CDELT1
CRPIX1
CUNIT1
CTYPE2
CRVAL2
CDELT2
CRPIX2
CUNIT2
CTYPE3
CRVAL3
CDELT3
CRPIX3
CUNIT3
CTYPE4
CRVAL4
CDELT4
CRPIX4
CUNIT4
PV2_1
PV2_2
TELESCOP
DATE-OBS
TIMESYS
OBSGEO-X
OBSGEO-Y
OBSGEO-Z
DATE
ORIGIN
TELECOP
PROJECT
SBID
DURATION
COMMENT
COMMENT
COMMENT
COMMENT
COMMENT
COMMENT
COMMENT
BMAJ
BMIN
BPA
FIELD
FIELDRA
FIELDDEC
WCSAXES
MJDREF
MJD-OBS
HISTORY
HISTORY
HISTORY
HISTORY
HISTORY
HISTORY
HISTORY
HISTORY
HISTORY
